## Search Engine With Tools and Agents


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
## Arxiv--Research
## Tool Creation
from langchain_community import tools
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [ ]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=300)

wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [18]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

##### Custom Tool[Rag Tools]

In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
web_path = "https://python.langchain.com/docs/tutorials/agents/" 
loader = WebBaseLoader(web_path=web_path)

embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
)
docs = loader.load()
document = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(document, embedding)
retriver = vectorstore.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7196879ca6f0>, search_kwargs={})

In [21]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriver,
    name="WebRetriever",
    description="Useful for retrieving information from the LangChain documentation website.",
)
retriever_tool.name

'WebRetriever'

In [22]:
tools = [wiki, arxiv, retriever_tool]

##### Running Tools with Agents and LLM Models

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash"
)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x719687503d70>, default_metadata=(), model_kwargs={})

In [24]:
## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
## Agents
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [26]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [28]:
agent_executer.invoke({"input": "What is LangChain?"})['output']



> Entering new AgentExecutor chain...

Invoking: `WebRetriever` with `{'query': 'LangChain'}`


Build an Agent | ğŸ¦œï¸�ğŸ”— LangChain

tipIn many applications, you may want to define custom tools. LangChain supports custom
tool creation via Python functions and other means. Refer to the
How to create tools guide for details.
Using Language Modelsâ€‹
Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

LangChain?EcosystemğŸ¦œğŸ› ï¸� LangSmithğŸ¦œğŸ•¸ï¸� LangGraphVersionsv0.3v0.2Pydantic compatibilityMigrating from v0.0 chainsHow to migrate from v0.0 chainsMigrating from ConstitutionalChainMigrating from ConversationalChainMigrating from ConversationalRetrievalChainMigrating from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigratin

'LangChain is a framework that allows you to build applications with large language models. It provides tools and functionalities for creating agents that can interact with language models, define custom tools, and manage memory for conversational interactions. You can find more detailed information and tutorials on building agents and other features within the LangChain documentation.'

In [29]:
agent_executer.invoke({'input':"what is machine learning?"})['output']



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdisciplineMachine learning is a field in artificial intelligence that focuses on creating and studying statistical algorithms. These algorithms can learn from data, apply that learning to new data, and perform tasks without needing specific instructions.

> Finished chain.


'Machine learning is a field in artificial intelligence that focuses on creating and studying statistical algorithms. These algorithms can learn from data, apply that learning to new data, and perform tasks without needing specific instructions.'

In [30]:
agent_executer.invoke({'input':"What is this research paper about Attention is all you need"})['output']



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Attention is all you need'}`


Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrocketed in recent months, and serving
models with I found a paper titled "Attention Is All You Need But You Don\'t Need All Of It For Inference of Large Language Models" published on July 22, 2024. It's about the inference demand for Large Language Models.

Are you referring to this paper, or the original "Attention Is All You Need" paper that introduced the Transformer architecture?

> Finished chain.


'I found a paper titled "Attention Is All You Need But You Don\\\'t Need All Of It For Inference of Large Language Models" published on July 22, 2024. It\'s about the inference demand for Large Language Models.\n\nAre you referring to this paper, or the original "Attention Is All You Need" paper that introduced the Transformer architecture?'